In [ ]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

In [ ]:
import pandas as pd 

df = pd.read_csv('https://raw.githubusercontent.com/reemalfarwan/nadeem/main/Datasets/ar_poem_metres.csv')

In [ ]:
df

In [ ]:
%%time 
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer


path = 'https://raw.githubusercontent.com/reemalfarwan/nadeem/main/Datasets/poems.csv'
# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

vocab_size=52000
# Customize training
tokenizer.train(files=path, vocab_size=vocab_size, min_frequency=5, special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>",])

In [ ]:
tokenizer.save_model(".", 'Model/')


['/content/gdrive/MyDrive/Model/-vocab.json',
 '/content/gdrive/MyDrive/Model/-merges.txt']

In [ ]:
!nvidia-smi


In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import RobertaTokenizerFast


tkn_path = 'Model/'
tokenizer = RobertaTokenizerFast.from_pretrained(tkn_path, max_len=512)


In [ ]:
from transformers import RobertaConfig
#my approach since march
config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=512,
    num_attention_heads=12,
    num_hidden_layers=12,
    type_vocab_size=1,
)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config).to('cuda')

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

datasets = load_dataset('csv', data_files='https://raw.githubusercontent.com/reemalfarwan/nadeem/main/Datasets/poems.csv')


In [ ]:
def tokenize_function(examples):
    #print(tokenizer(examples["text"]))
    return tokenizer(examples["poem_text"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["poem_text"])

In [ ]:
tokenized_datasets

In [ ]:
block_size = 128

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="Model/epoch_10",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_gpu_train_batch_size=32,
    save_steps=20000,
    save_total_limit=2,
    prediction_loss_only=True,
    
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],

    
)


In [ ]:
%%time

trainer.train()  

In [ ]:
trainer.save_model("Model/RoBERTa_poem")
